In [3]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\Users\Ilya Jeff\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [86]:
import seaborn as sns
import glob
import torch
import os
import numpy as np

import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
sns.set(style="whitegrid")
mpl.rcParams['agg.path.chunksize'] = 10000

#from matplotlib import rc
#rc('text', usetex=True)
mpl.rcParams['axes.titlesize'] = 'xx-large'
mpl.rcParams['axes.labelsize'] = 'xx-large'


In [69]:
rerun_rounds = ['round1', 'round2', 'round3']
def get_avg_results(path_string):
  """
  For experiments with multiple random training runs, collects all
  training runs and averages their results
  """
  result_list = []
  for rerun_string in rerun_rounds:
    completed_path = path_string.format(rerun_string)
    if os.path.exists(completed_path):
      result_list.append(float(open(completed_path).read()))
  return sum(result_list)/len(result_list) if len(result_list) > 0 else -1.0
  #print(result_list)
  #return np.std(result_list) if len(result_list) > 0 else -1 #sum(result_list)/len(result_list) if len(result_list) > 0 else -1.0

In [70]:
#a = torch.tensor([get_avg_results('reports/dep_rel-6-768-en-distilbert-layer{}.yaml.results/dev.v_entropy'.format(i)) for i in range(1,7)])
#b = torch.tensor([float(open('reports/dep_rel-6-768-en-distilbert-layer{}.yaml.results/dev.v_entropy'.format(i)).read().strip()) for i in range(1,7)])
#c = get_avg_results('reports/layer0.yaml.results/dev.v_entropy')
#print(a)
#print(b)

#for i in range(1,7):
#   print(i-1)    

In [83]:
def plot_layerwise_zero_baseline(task, model):
    fig, ax = plt.subplots(figsize=(6,4.5))
    ax.tick_params(axis='both', which='major', labelsize=15)
    ax.tick_params(axis='both', which='minor', labelsize=15)
    #change depending on layers
    ax.set_xticks(list(range(1,9)))
    """Makes a plot comparing layerwise bits of info"""
    #change range depending on layers of model
    task_768_singlelayer_ventropy = torch.tensor([get_avg_results('reports/{}-{}-layer{}.yaml.results/dev.v_entropy'.format(task,model,i)) for i in range(1,9)])
    task_768_twolayer_ventropy = torch.tensor([get_avg_results('reports/{}-{}-layer{}-0.yaml.results/dev.v_entropy'.format(task,model,i)) for i in range(1,9)])
    task_768_embedding_layer_ventropy = get_avg_results('reports/{}-{}-layer0.yaml.results/dev.v_entropy'.format(task,model))

    #print(task_768_singlelayer_ventropy)
    #print(task_768_twolayer_ventropy)
    #print(task_768_embedding_layer_ventropy)

    #12 layer
    #x_range = list(range(1,13))

    #6 layer
    #x_range = list(range(1,7))

    #3 layer
    #x_range = list(range(1,4))

    #8 layer
    x_range = list(range(1,9))

    #10 layer
    #x_range = list(range(1,11))

    plt.plot(x_range, task_768_embedding_layer_ventropy-task_768_singlelayer_ventropy, label=r'baselined')
    plt.plot(x_range, task_768_embedding_layer_ventropy-task_768_twolayer_ventropy, label=r'conditional')

    #plot title
    plt.title('{} {}'.format(model,task), fontsize = 12)

    plt.ylabel('Bits', fontsize = 12)
    plt.xlabel('Model layer', fontsize = 12)
    plt.legend()
    plt.tight_layout()

    #save figure in folder
    plt.savefig('graphs/{0}/{0} {1}.png'.format(model,task), dpi=200)

    plt.clf()

In [82]:
def plot_accuracy(task, model):
    fig, ax = plt.subplots(figsize=(6,4.5))
    ax.tick_params(axis='both', which='major', labelsize=15)
    ax.tick_params(axis='both', which='minor', labelsize=15)
    #change depending on layers
    ax.set_xticks(list(range(1,9)))
    """Makes a plot accuracy"""
    #change range depending on layers of model
    label_accuracy = torch.tensor([get_avg_results('reports/{}-{}-layer{}.yaml.results/dev.label_acc'.format(task,model,i)) for i in range(1,9)])

    #print(label_accuracy)

    #12 layer
    #x_range = list(range(1,13))

    #6 layer
    #x_range = list(range(1,7))

    #3 layer
    #x_range = list(range(1,4))

    #8 layer
    x_range = list(range(1,9))

    #10 layer
    #x_range = list(range(1,11))

    plt.plot(x_range, label_accuracy)

    #plot title
    plt.title('{} {} accuracy'.format(model,task), fontsize = 12)

    plt.ylabel('Accuracy', fontsize = 12)
    plt.xlabel('Model layer', fontsize = 12)
    plt.tight_layout()

    #save figure in folder
    plt.savefig('graphs/{0}/{0} {1} accuracy.png'.format(model,task), dpi=200)

    plt.clf()

In [113]:
def plot_accuracy_combined(task):
    fig, ax = plt.subplots(figsize=(10,4.5))
    ax.tick_params(axis='both', which='major', labelsize=15)
    ax.tick_params(axis='both', which='minor', labelsize=15)
    #change depending on layers
    ax.set_xticks(list(range(1,13)))
    """Makes a plot comparing layerwise bits of info"""
    #bert768
    label_accuracy_bert768 = torch.tensor([get_avg_results('reports/{}-bert768-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,13)])

    #bert-l2-prune30
    label_accuracy_bert_l2_prune30 = torch.tensor([get_avg_results('reports/{}-bert-l2-prune30-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,13)])

    #distilbert-base-uncased
    label_accuracy_distilbert_base_uncased = torch.tensor([get_avg_results('reports/{}-distilbert-base-uncased-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,7)])

    #6-768-distilbert
    label_accuracy_6_768 = torch.tensor([get_avg_results('reports/{}-6-768-en-distilbert-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,7)])

    #6-b128-distilbert
    label_accuracy_6_b128 = torch.tensor([get_avg_results('reports/{}-6-b128-distilbert-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,7)])

    #6-768-b32-distilbert
    label_accuracy_6_768_b32 = torch.tensor([get_avg_results('reports/{}-6-768-b32-distilbert-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,7)])

    #3-768-distilbert
    label_accuracy_3_768 = torch.tensor([get_avg_results('reports/{}-3-768-en-distilbert-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,4)])

    #8-768-distilbert
    label_accuracy_8_768 = torch.tensor([get_avg_results('reports/{}-8-768-en-distilbert-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,9)])

    #8-768-s9273-distilbert
    label_accuracy_8_768_s9273 = torch.tensor([get_avg_results('reports/{}-8-768-en-s9273-distilbert-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,9)])

    #10-768-distilbert
    label_accuracy_10_768 = torch.tensor([get_avg_results('reports/{}-10-768-en-distilbert-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,11)])

    #10-b128-distilbert
    label_accuracy_10_b128 = torch.tensor([get_avg_results('reports/{}-10-b128-distilbert-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,11)])

    #12-768-distilbert
    label_accuracy_12_768 = torch.tensor([get_avg_results('reports/{}-12-768-en-distilbert-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,13)])

    #MiniLMv2-L6-H768-distilled-from-BERT-Base
    label_accuracy_minilmv2 = torch.tensor([get_avg_results('reports/{}-MiniLMv2-L6-H768-distilled-from-BERT-Base-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,7)])

    #print(label_accuracy)

    #12 layer
    x_range1 = list(range(1,13))
    #3 layer
    x_range2 = np.linspace(1,12,3)
    #6 layer
    x_range3 = np.linspace(1,12,6)
    #8 layer
    x_range4 = np.linspace(1,12,8)
    #10 layer
    x_range5 = np.linspace(1,12,10)

    #plt.plot(x_range1, label_accuracy_bert768, label=r'bert_768')
    #plt.plot(x_range3, label_accuracy_distilbert_base_uncased, label=r'distilbert-base-uncased')
    #plt.plot(x_range3, label_accuracy_minilmv2, label=r'MiniLMv2')
    #plt.plot(x_range1, label_accuracy_bert_l2_prune30, label=r'bert-l2-prune30')
    plt.plot(x_range2, label_accuracy_3_768, label=r'3-768-distilbert')
    plt.plot(x_range3, label_accuracy_6_768, label=r'6-768-distilbert')
    plt.plot(x_range3, label_accuracy_6_768_b32, label=r'6-768-b32-distilbert')
    plt.plot(x_range3, label_accuracy_6_b128, label=r'6-b128-distilbert')
    plt.plot(x_range4, label_accuracy_8_768, label=r'8-768-distilbert')
    plt.plot(x_range4, label_accuracy_8_768_s9273, label=r'8-768-s9273-distilbert')
    plt.plot(x_range5, label_accuracy_10_768, label=r'10-768-distilbert')
    plt.plot(x_range5, label_accuracy_10_b128, label=r'10-b128-distilbert')
    plt.plot(x_range1, label_accuracy_12_768, label=r'12-768-distilbert')

    plt.title('Accuracy of {} for variants of bert scaled for 12 layers'.format(task), fontsize = 12)

    plt.ylabel('Accuracy', fontsize = 12)
    plt.xlabel('Model layer', fontsize = 12)
    #plt.legend()
    plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left', fontsize = "small")
    plt.tight_layout()

    #plt.savefig('graphs/Accuracies for {} bert768 variants scaled to 12 layers.png'.format(task), dpi=200)
    plt.savefig('graphs/Accuracies for {} wo bert768.png scaled to 12 layers.png'.format(task), dpi=200)

    plt.clf()

In [115]:
def plot_conditional_combined(task):
    fig, ax = plt.subplots(figsize=(10,4.5))
    ax.tick_params(axis='both', which='major', labelsize=15)
    ax.tick_params(axis='both', which='minor', labelsize=15)
    #change depending on layers
    ax.set_xticks(list(range(1,13)))
    """Makes a plot comparing layerwise bits of info"""
    #bert768
    twolayer_bert768 = torch.tensor([get_avg_results('reports/{}-bert768-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,13)])
    embedded1 = get_avg_results('reports/{}-bert768-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_bert768 = embedded1 - twolayer_bert768

    #bert-l2-prune30
    twolayer_bertl2 = torch.tensor([get_avg_results('reports/{}-bert-l2-prune30-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,13)])
    embedded2 = get_avg_results('reports/{}-bert-l2-prune30-layer0.yaml.results/dev.v_entropy'.format(task))
    conditiona_l2 = embedded2 - twolayer_bertl2

    #distilbert-base-uncased
    twolayer_distil = torch.tensor([get_avg_results('reports/{}-distilbert-base-uncased-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,7)])
    embedded3 = get_avg_results('reports/{}-distilbert-base-uncased-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_distil = embedded3 - twolayer_distil

    #6-768-distilbert
    twolayer_6_768 = torch.tensor([get_avg_results('reports/{}-6-768-en-distilbert-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,7)])
    embedded4 = get_avg_results('reports/{}-6-768-en-distilbert-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_6_768 = embedded4 - twolayer_6_768

    #6-b128-distilbert
    twolayer_6_b128 = torch.tensor([get_avg_results('reports/{}-6-b128-distilbert-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,7)])
    embedded5 = get_avg_results('reports/{}-6-b128-distilbert-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional5_6_b128 = embedded5 - twolayer_6_b128

    #6-768-b32-distilbert
    twolayer_6_b32 = torch.tensor([get_avg_results('reports/{}-6-768-b32-distilbert-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,7)])
    embedded6 = get_avg_results('reports/{}-6-768-b32-distilbert-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_6_b32 = embedded6 - twolayer_6_b32

    #3-768-distilbert
    twolayer_3_768 = torch.tensor([get_avg_results('reports/{}-3-768-en-distilbert-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,4)])
    embedded7 = get_avg_results('reports/{}-3-768-en-distilbert-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_3_768 = embedded7 - twolayer_3_768

    #8-768-distilbert
    twolayer_8_768 = torch.tensor([get_avg_results('reports/{}-8-768-en-distilbert-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,9)])
    embedded8 = get_avg_results('reports/{}-8-768-en-distilbert-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_8_768 = embedded8 - twolayer_8_768

    #8-768-s9273-distilbert
    twolayer_8_768_S9273 = torch.tensor([get_avg_results('reports/{}-8-768-en-s9273-distilbert-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,9)])
    embedded9 = get_avg_results('reports/{}-8-768-en-s9273-distilbert-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_8_s9273 = embedded9 - twolayer_8_768_S9273

    #10-768-distilbert
    twolayer_10_768 = torch.tensor([get_avg_results('reports/{}-10-768-en-distilbert-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,11)])
    embedded10 = get_avg_results('reports/{}-10-768-en-distilbert-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_10_768 = embedded10 - twolayer_10_768

    #10-b128-distilbert
    twolayer_10_b128 = torch.tensor([get_avg_results('reports/{}-10-b128-distilbert-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,11)])
    embedded11 = get_avg_results('reports/{}-10-b128-distilbert-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_10_b128 = embedded11 - twolayer_10_b128

    #12-768-distilbert
    twolayer_12_768 = torch.tensor([get_avg_results('reports/{}-12-768-en-distilbert-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,13)])
    embedded12 = get_avg_results('reports/{}-12-768-en-distilbert-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_12_768 = embedded12 - twolayer_12_768

    #MiniLMv2-L6-H768-distilled-from-BERT-Base
    twolayer_minilmv2 = torch.tensor([get_avg_results('reports/{}-MiniLMv2-L6-H768-distilled-from-BERT-Base-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,7)])
    embedded13 = get_avg_results('reports/{}-MiniLMv2-L6-H768-distilled-from-BERT-Base-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_minilmv2 = embedded13 - twolayer_minilmv2

    #print(label_accuracy)

    #12 layer
    x_range1 = list(range(1,13))
    #3 layer
    x_range2 = np.linspace(1,12,3)
    #6 layer
    x_range3 = np.linspace(1,12,6)
    #8 layer
    x_range4 = np.linspace(1,12,8)
    #10 layer
    x_range5 = np.linspace(1,12,10)

    plt.plot(x_range1, conditional_bert768, label=r'bert_768')
    plt.plot(x_range3, conditional_distil, label=r'distilbert-base-uncased')
    plt.plot(x_range3, conditional_minilmv2, label=r'MiniLMv2')
    plt.plot(x_range1, conditiona_l2, label=r'bert-l2-prune30')
    plt.plot(x_range2, conditional_3_768, label=r'3-768-distilbert')
    plt.plot(x_range3, conditional_6_768, label=r'6-768-distilbert')
    plt.plot(x_range3, conditional_6_b32, label=r'6-768-b32-distilbert')
    plt.plot(x_range3, conditional5_6_b128, label=r'6-b128-distilbert')
    plt.plot(x_range4, conditional_8_768, label=r'8-768-distilbert')
    plt.plot(x_range4, conditional_8_s9273, label=r'8-768-s9273-distilbert')
    plt.plot(x_range5, conditional_10_768, label=r'10-768-distilbert')
    plt.plot(x_range5, conditional_10_b128, label=r'10-b128-distilbert')
    plt.plot(x_range1, conditional_12_768, label=r'12-768-distilbert')

    plt.title('V-Entropy of {} for variants of bert scaled for 12 layers'.format(task), fontsize = 12)

    plt.ylabel('Accuracy', fontsize = 12)
    plt.xlabel('Model layer', fontsize = 12)
    #plt.legend()
    plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left', fontsize = "small")
    plt.tight_layout()

    plt.savefig('graphs/V-Entropy for {} bert768 variants scaled to 12 layers.png'.format(task), dpi=200)
    #plt.savefig('graphs/V-Entropy for {} wo bert768.png scaled to 12 layers.png'.format(task), dpi=200)

    plt.clf()

In [84]:
plot_layerwise_zero_baseline('dep_rel','8-b128-distilbert')
plot_layerwise_zero_baseline('named_entities','8-b128-distilbert')
plot_layerwise_zero_baseline('ptb_pos','8-b128-distilbert')
plot_layerwise_zero_baseline('upos','8-b128-distilbert')
plot_layerwise_zero_baseline('word_sense','8-b128-distilbert')
plot_layerwise_zero_baseline('sst2','8-b128-distilbert')


In [85]:
plot_accuracy('dep_rel','8-b128-distilbert')
plot_accuracy('named_entities','8-b128-distilbert')
plot_accuracy('ptb_pos','8-b128-distilbert')
plot_accuracy('upos','8-b128-distilbert')
plot_accuracy('word_sense','8-b128-distilbert')
plot_accuracy('sst2','8-b128-distilbert')


In [114]:
plot_accuracy_combined('dep_rel')
plot_accuracy_combined('named_entities')
plot_accuracy_combined('ptb_pos')
plot_accuracy_combined('upos')
plot_accuracy_combined('word_sense')
plot_accuracy_combined('sst2')


In [116]:
plot_conditional_combined('dep_rel')
plot_conditional_combined('named_entities')
plot_conditional_combined('ptb_pos')
plot_conditional_combined('upos')
plot_conditional_combined('word_sense')
plot_conditional_combined('sst2')

ValueError: x and y must have same first dimension, but have shapes (6,) and torch.Size([8])

In [ ]:
#tests
def plot_layerwise_adjacent(task, model):
    fig, ax = plt.subplots(figsize=(6,4.5))
    ax.tick_params(axis='both', which='major', labelsize=15)
    ax.tick_params(axis='both', which='minor', labelsize=15)
    #change depending on layers
    ax.set_xticks(list(range(1,7)))
    """Makes a plot comparing layerwise bits of info"""
    #change range depending on layers of model
    task_768_singlelayer_ventropy = torch.tensor([get_avg_results('reports/{}-{}-layer{}.yaml.results/dev.v_entropy'.format(task,model,i)) for i in range(1,7)])
    task_768_twolayer_ventropy = torch.tensor([get_avg_results('reports/{}-{}-layer{}-{}.yaml.results/dev.v_entropy'.format(task,model,i-1,i)) for i in range(1,7)])
    task_768_embedding_layer_ventropy = torch.tensor([get_avg_results('reports/{}-{}-layer{}.yaml.results/dev.v_entropy'.format(task,model,i-1)) for i in range(1,7)])

    #print(task_768_singlelayer_ventropy)
    #print(task_768_twolayer_ventropy)
    #print(task_768_embedding_layer_ventropy)

    #12 layer
    #x_range = list(range(1,13))

    #6 layer
    x_range = list(range(1,7))

    #3 layer
    #x_range = list(range(1,4))

    #8 layer
    #x_range = list(range(1,9))

    #10 layer
    #x_range = list(range(1,11))

    plt.plot(x_range, task_768_embedding_layer_ventropy-task_768_singlelayer_ventropy, label=r'baselined')
    plt.plot(x_range, task_768_embedding_layer_ventropy-task_768_twolayer_ventropy, label=r'conditional')

    #plot title
    plt.title('{} {}'.format(model,task), fontsize = 12)

    plt.ylabel('Bits', fontsize = 12)
    plt.xlabel('Model layer', fontsize = 12)
    plt.legend()
    plt.tight_layout()

    #save figure in folder
    plt.savefig('graphs/{0}/{0} {1} test.png'.format(model,task), dpi=200)

    plt.clf()

In [ ]:
plot_layerwise_adjacent('dep_rel','MiniLMv2-L6-H768-distilled-from-BERT-Base')
